# 自定义激活函数、初始化、正则化和约束

```{note}
很简单，直接用函数自定义即可，当然也可以子类定义。
```

## 函数定义

分别举例说明

In [1]:
import tensorflow as tf
from tensorflow import keras


def my_softplus(z):
    # 自定义激活函数
    return tf.math.log(tf.exp(z) + 1.0)

In [2]:
def my_glorot_initializer(shape, dtype=tf.float32):
    # 自定义初始化
    # glorot初始化就是xavier初始化
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

In [3]:
def my_l1_regularizer(weights):
    # 自定义正则化
    return tf.reduce_sum(tf.abs(0.01 * weights))

In [4]:
def my_positive_weights(weights):
    # 自定义约束
    # 效果等同于 tf.nn.relu(weights)
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [5]:
# 用起来没有什么不同
layer = keras.layers.Dense(1, activation=my_softplus,
                           kernel_initializer=my_glorot_initializer,
                           kernel_regularizer=my_l1_regularizer,
                           kernel_constraint=my_positive_weights)

## 子类定义

以正则化为例进行说明，它继承自 keras.regularizers.Regularizer

In [6]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
        
    def __call__(self, weights):
        # 正则化、初始化和约束实现 __call__() 方法
        # 层、损失函数和激活函数实现 call() 方法
        return tf.reduce_sum(tf.abs(self.factor * weights))
    
    def get_config(self):
        return {"factor": self.factor}